# Luiata Data Science Take-home Assignment - Part I: Data Processing

### Jade Yun | [LinkedIn](https://www.linkedin.com/in/jadeyun/) | [GitHub](https://github.com/yuyun2)

## Overview

This notebook contains code for:
- Data ETL
- Feature Generation

### 1. Data ETL
Raw data is stored in JSON files. To perform data analysis and modeling, we need to extract data into tabular forms.

We will use python built-in SQLlite to generate features.

Here are the steps of data processing:

1. Parse JSON file into csv
2. Read csv into Pandas DataFrame
3. Write dataframe into SQL database

### 2. Feature Generation
Features are generated on patients level. There are 3 main feature categories:
- Count Features:   
- Days Features
- Lab value Features

#### Count Features
Count number of doctor visits / lab tests / procedures, combine with time window (e.g. # of doctor visits within 6 moths).
#### Days Fetures
Count number of days since first/last doctor visits / lab tests / procedures (e.g. # of days since last doctor visits)
#### Lab value Features
- Calculate average values of selected lab tests (e.g. avg value of patient's lab test loinc_27353-2).
- Find values of last selected lab tests(e.g. value of patient's most recent lab test loinc_4548-4)

In [1]:
import json
import pandas as pd
from pprint import pprint

import sys
import csv

from sqlalchemy import create_engine

## Data ETL
### JSON to CSV

Use IOstream to process JSON file line by line and write into csv file

In [2]:
def json_to_csv(files):
    """ convert json data to csv format
        input: list of files in json format
        return: none
    """
    # open three csv files to write
    with open('basic_info.csv', 'w') as basic_info,\
         open('resource.csv', 'w') as resource,\
         open('observation.csv', 'w') as observation:
    
        writer_basic = csv.writer(basic_info, lineterminator='\n') # initiate a csv writer
        writer_basic.writerow(['patient_id', 'birthday', 'is_male', 'tag_dm2', 'split']) # write a header
        total_pat = 0 # initiate a count for a csv file
        
        writer_res = csv.writer(resource, lineterminator='\n')
        writer_res.writerow(['patient_id', 'visit_date', 'resource_code'])
        total_res = 0 
        
        write_ob = csv.writer(observation, lineterminator='\n')
        write_ob.writerow(['patient_id', 'lab_date', 'loinc_code', 'interpretation', 'value'])
        total_ob = 0
        
        # loop through files
        for file in files:
            print("processing file {}...".format(file))
            
            # initiate counter for each table within a file
            cnt_pat, cnt_res, cnt_ob = 0, 0, 0
            
            # process each line of the json file
            with open(file) as f:
                
                # extract basic information for each patient 
                for line in f:
                    patient = json.loads(line)
                    pat_id = patient['patient_id']
                    bday =  patient['bday']
                    is_male = int(patient['is_male'])
                    tag_dm2 = patient['tag_dm2']
                    split = patient['split']

                    writer_basic.writerow([pat_id, bday, is_male, tag_dm2, split])
                    cnt_pat += 1
                    total_pat += 1
                    
                    # extract resources information for each patient
                    for visit_date, resources in patient['resources'].items():
                        if len(resources) == 0:
                            writer_res.writerow([pat_id, visit_date, 'None'])
                            cnt_res += 1
                            total_res += 1
                        else:
                            for res in resources:
                                writer_res.writerow([pat_id, visit_date, res])
                                cnt_res += 1
                                total_res += 1
                                
                    # extract observation information for each patient
                    for ob_date, observations in patient['observations'].items():
                        for ob in observations:
                            lab_code = ob['code']
                            interpret = ob['interpretation']
                            val = ob['value']
                            write_ob.writerow([pat_id, ob_date, lab_code, interpret, val])  
                            cnt_ob += 1
                            total_ob += 1
            print("{}: processed {} patients, {} resources, {} observations".format(file, cnt_pat, cnt_res, cnt_ob)) 
                            
        print("finished, file output to current directory")
        print("total: {} in basic_info, {} in resources, {} in observations".format(total_pat, total_res, total_ob))

In [167]:
# convert train and text from json to csv
files_to_process = ['train.txt', 'test.txt']
json_to_csv(files_to_process)

processing file train.txt...
train.txt: processed 73597 patients, 12526083 resources, 4169499 observations
processing file test.txt...
test.txt: processed 31403 patients, 3929457 resources, 1257280 observations
finished, file output to current directory
total: 105000 in basic_info, 16455540 in resources, 5426779 in observations


### CSV to SQL

First read csv into Pandas Dataframe, then use Pandas dataframe's method to store into SQL database

In [168]:
def csv_to_sql(files, db_engine):
    """ insert csv data into sqlite database
        input: list of files to process
        return: None
    """ 
    
    # loop through files
    for file in files:
        
        print("inserting {}...".format(file))
        
        table = file.replace('.csv','') #
        
        df = pd.read_csv(file)
        df.to_sql(table, db_engine, index=False, if_exists='replace', chunksize=10**4)
    
        print('inserted {} rows into table {}'.format(df.shape[0], table))

In [169]:
# insert basic_info, resources and observation into sqlite
files_to_insert = ['basic_info.csv', 'resource.csv', 'observation.csv']

lumiata = create_engine('sqlite:///lumiata.db')
csv_to_sql(files_to_insert, lumiata)

inserting basic_info.csv...
inserted 105000 rows into table basic_info
inserting resource.csv...
inserted 16455540 rows into table resource
inserting observation.csv...
inserted 5426779 rows into table observation


## Feature Engineering
### Resource Data
Count features:
- number of visit
- number of visit in time window (most recent 1/2/3/6/12 months)
- number of visit with resource code
- number of unique resource code
- number of visit with resource code in time window
- number of resource code by category 
- number of resource code weighted by time, give recent vist more weight (not performed because SQLlite doesn't have built-in math functions like exponential) 

In [3]:
# create database connection
lumiata = create_engine('sqlite:///lumiata.db')

In [4]:
pd.read_sql_query('SELECT * FROM resource LIMIT 5', lumiata)

,patient_id,visit_date,resource_code
0,pat_0,2016-08-10,None
1,pat_0,2016-07-13,None
2,pat_0,2017-06-10,None
3,pat_0,2014-01-13,None
4,pat_0,2014-04-30,None


In [167]:
lumiata.execute('drop table if exists res_num_visit')

In [169]:
# generate features using the resource table

query = """
Create table res_num_visit as 

Select patient_id, 

       -- count total/unique resource
       sum(case when resource_code <> 'None' then 1 else 0 end) as num_total_res,
       count(distinct resource_code) as num_uniq_res,
       
       -- count numbers of icd procedures with/without time window
       sum(case when resource_code like 'icd%' then 1 else 0 end) as num_icd,
       sum(case when resource_code like 'icd%' and visit_date >= '2016-12-01' then 1 else 0 end) as num_icd_1mo, 
       sum(case when resource_code like 'icd%' and visit_date >= '2016-11-01' then 1 else 0 end) as num_icd_2mo,
       sum(case when resource_code like 'icd%' and visit_date >= '2016-10-01' then 1 else 0 end) as num_icd_3mo, 
       sum(case when resource_code like 'icd%' and visit_date >= '2016-07-01' then 1 else 0 end) as num_icd_6mo,
       sum(case when resource_code like 'icd%' and visit_date >= '2016-01-01' then 1 else 0 end) as num_icd_12mo,  
       
       -- count numbers of cpt procedures with/without time window
       sum(case when resource_code like 'cpt%' then 1 else 0 end) as num_cpt,
       sum(case when resource_code like 'cpt%' and visit_date >= '2016-12-01' then 1 else 0 end) as num_cpt_1mo, 
       sum(case when resource_code like 'cpt%' and visit_date >= '2016-11-01' then 1 else 0 end) as num_cpt_2mo,
       sum(case when resource_code like 'cpt%' and visit_date >= '2016-10-01' then 1 else 0 end) as num_cpt_3mo, 
       sum(case when resource_code like 'cpt%' and visit_date >= '2016-07-01' then 1 else 0 end) as num_cpt_6mo,
       sum(case when resource_code like 'cpt%' and visit_date >= '2016-01-01' then 1 else 0 end) as num_cpt_12mo,
       
       -- count numbers of visit with/without time window
       count(distinct visit_date) as num_visit,
       count(distinct case when visit_date >= '2016-12-01' then visit_date else null end) as num_visit_1mo, 
       count(distinct case when visit_date >= '2016-11-01' then visit_date else null end) as num_visit_2mo,
       count(distinct case when visit_date >= '2016-10-01' then visit_date else null end) as num_visit_3mo,
       count(distinct case when visit_date >= '2016-07-01' then visit_date else null end) as num_visit_6mo,
       count(distinct case when visit_date >= '2016-01-01' then visit_date else null end) as num_visit_12mo,
       
       -- count numbers of visit with procedure and with/without time window
       count(distinct case when resource_code <> 'None' then visit_date else null end) as num_visit_res,
       count(distinct case when resource_code <> 'None' and visit_date >= '2016-12-01' then visit_date else null end) as num_visit_res_1mo, 
       count(distinct case when resource_code <> 'None' and visit_date >= '2016-11-01' then visit_date else null end) as num_visit_res_2mo,
       count(distinct case when resource_code <> 'None' and visit_date >= '2016-10-01' then visit_date else null end) as num_visit_res_3mo,
       count(distinct case when resource_code <> 'None' and visit_date >= '2016-07-01' then visit_date else null end) as num_visit_res_6mo,
       count(distinct case when resource_code <> 'None' and visit_date >= '2016-01-01' then visit_date else null end) as num_visit_res_12mo
    
       
From resource 
Where visit_date < '2017-01-01'
Group by patient_id
"""
lumiata.execute(query)

In [54]:
pd.read_sql_query('SELECT * FROM res_num_visit LIMIT 5', lumiata)

,patient_id,num_total_res,num_uniq_res,num_icd,num_icd_1mo,num_icd_2mo,num_icd_3mo,num_icd_6mo,num_icd_12mo,num_cpt,...,num_visit_res_6mo,num_visit_res_12mo,d_first_visit,d_last_visit,d_first_res,d_last_res,d_first_icd,d_last_icd,d_first_cpt,d_last_cpt
0,pat_0,98,64,43,0,0,0,0,4,55,...,0,3,1083.0,7.0,955.0,258.0,955.0,258.0,955.0,258.0
1,pat_1,90,67,37,0,0,0,2,14,53,...,1,3,1116.0,151.0,1116.0,151.0,1116.0,151.0,1116.0,151.0
2,pat_10,94,80,36,0,0,0,5,5,58,...,1,1,1435.0,178.0,1435.0,178.0,1435.0,178.0,1435.0,178.0
3,pat_100,96,66,57,8,8,8,18,21,39,...,4,6,938.0,17.0,938.0,17.0,938.0,17.0,938.0,17.0
4,pat_1000,151,81,91,0,2,16,41,62,60,...,14,17,1083.0,28.0,1072.0,53.0,1072.0,53.0,1072.0,53.0


### Check Claim Rate by Resource Code

Since there are more then 30K unique resource code, we need to identify the codes that are more related with diabetes.

Here we calcuate the claim rate for each resource code (# patient with the code who claimed / # of patient with the code)

For each resource code, number of patient that has this code is also important, we want to use more common code as features.

And then generate features for top claim-rate resource codes

- claim rate for resource code (this is not directly used as features to prevent data leakge)
- count of selected code 

In [88]:
lumiata.execute('drop table if exists res_freq')

In [90]:
# generate features using the resource table

query = """
Create table res_freq as 

-- claim rate for each resource code

Select resource_code, 
       count(distinct r.patient_id) as num_pat,
       count(distinct case when b.tag_dm2 is not null then b.patient_id else null end) as cnt_claim_pat,
       count(distinct case when b.tag_dm2 is not null then b.patient_id else null end)*1.0 / count(distinct r.patient_id) as claim_rate
        
From resource r 
left join basic_info b on r.patient_id = b.patient_id

Where visit_date < '2017-01-01'
      and resource_code <> 'None'
      and b.split = 'train'
      and (b.tag_dm2 <= '2017-12-31' or b.tag_dm2 is null)
      
Group by resource_code
order by cnt_claim_pat desc
"""
lumiata.execute(query)

In [91]:
pd.read_sql_query('SELECT count(1) as cnt_res_code FROM res_freq ', lumiata)

,cnt_res_code
0,35871


In [208]:
pd.read_sql_query('SELECT * FROM res_freq where num_pat > 5000 order by claim_rate desc LIMIT 10', lumiata)

,resource_code,num_pat,cnt_claim_pat,claim_rate
0,cpt_83036,8720,884,0.101376
1,icd9_401.1,7666,777,0.101357
2,icd10_I10,16947,1658,0.097834
3,icd10_E78.5,9319,901,0.096684
4,icd9_401.9,13016,1258,0.096650
5,cpt_99223,5333,509,0.095443
6,cpt_93306,7738,718,0.092789
7,cpt_71010,5997,548,0.091379
8,icd9_272.4,11860,1069,0.090135
9,cpt_99232,5790,521,0.089983


Show resource code with lowest claim rate that has at least 5000 patients

In [209]:
pd.read_sql_query('SELECT * FROM res_freq where num_pat > 5000 order by claim_rate LIMIT 10', lumiata)

,resource_code,num_pat,cnt_claim_pat,claim_rate
0,icd10_Z00.129,7615,22,0.002889
1,icd9_V20.2,8234,26,0.003158
2,cpt_90460,5144,19,0.003694
3,icd10_Z71.89,5486,84,0.015312
4,icd10_Z71.3,6291,101,0.016055
5,cpt_87880,8748,147,0.016804
6,cpt_99395,7094,140,0.019735
7,icd10_J02.9,5380,124,0.023048
8,cpt_90686,6556,165,0.025168
9,icd9_462,6172,161,0.026086


### Count of Top Claim Rate Resource Code 
Show resource code with highest claim rate that has at least 5000 patients

In [106]:
lumiata.execute('drop table if exists res_one_hot')

In [107]:
# generate features using the resource table

query = """
Create table res_one_hot as 

Select patient_id, 

    -- count of resource code with high claim rate
    sum(case when resource_code = 'cpt_83036'     then 1 else 0 end) as cnt_cpt_83036,
    sum(case when resource_code = 'icd9_401.1'    then 1 else 0 end) as cnt_icd9_401_1,
    sum(case when resource_code = 'icd10_I10'     then 1 else 0 end) as cnt_icd10_I10,
    sum(case when resource_code = 'icd10_E78.5'   then 1 else 0 end) as cnt_icd10_E78_5,
    sum(case when resource_code = 'icd9_401.9'    then 1 else 0 end) as cnt_icd9_401_9,
    sum(case when resource_code = 'cpt_99223'     then 1 else 0 end) as cnt_cpt_99223,
    sum(case when resource_code = 'cpt_93306'     then 1 else 0 end) as cnt_cpt_93306,
    sum(case when resource_code = 'cpt_71010'     then 1 else 0 end) as cnt_cpt_71010,
    sum(case when resource_code = 'icd9_272.4'    then 1 else 0 end) as cnt_icd9_272_4,
    sum(case when resource_code = 'cpt_99232'     then 1 else 0 end) as cnt_cpt_99232,
    sum(case when resource_code = 'icd10_R73.09'  then 1 else 0 end) as cnt_icd10_R73_09,
    sum(case when resource_code = 'icd10_R73.01'  then 1 else 0 end) as cnt_icd10_R73_01,
    sum(case when resource_code = 'icd9_790.29'   then 1 else 0 end) as cnt_icd9_790_29,
    sum(case when resource_code = 'icd10_I50.9'   then 1 else 0 end) as cnt_icd10_I50_9,
    sum(case when resource_code = 'icd10_E66.01'  then 1 else 0 end) as cnt_icd10_E66_01,
    sum(case when resource_code = 'icd9_790.21'   then 1 else 0 end) as cnt_icd9_790_21,
    sum(case when resource_code = 'icd9_414.01'   then 1 else 0 end) as cnt_icd9_414_01,
    sum(case when resource_code = 'icd9_278.01'   then 1 else 0 end) as cnt_icd9_278_01,
    sum(case when resource_code = 'icd9_428.0'    then 1 else 0 end) as cnt_icd9_428_0,
    sum(case when resource_code = 'cpt_A7037'     then 1 else 0 end) as cnt_cpt_A7037,
    
    -- count of resource code with low claim rate
    sum(case when resource_code = 'icd10_Z00.129' then 1 else 0 end) as cnt_icd10_Z00_129,
    sum(case when resource_code = 'icd9_V20.2'    then 1 else 0 end) as cnt_icd9_V20_2,
    sum(case when resource_code = 'cpt_90460'     then 1 else 0 end) as cnt_cpt_90460,
    sum(case when resource_code = 'icd10_Z71.89'  then 1 else 0 end) as cnt_icd10_Z71_89,
    sum(case when resource_code = 'icd10_Z71.3'   then 1 else 0 end) as cnt_icd10_Z71_3,
    sum(case when resource_code = 'cpt_87880'     then 1 else 0 end) as cnt_cpt_87880,
    sum(case when resource_code = 'cpt_99395'     then 1 else 0 end) as cnt_cpt_99395,
    sum(case when resource_code = 'icd10_J02.9'   then 1 else 0 end) as cnt_icd10_J02_9,
    sum(case when resource_code = 'cpt_90686'     then 1 else 0 end) as cnt_cpt_90686,
    sum(case when resource_code = 'icd9_462'      then 1 else 0 end) as cnt_icd9_462,
    sum(case when resource_code = 'cpt_90685'     then 1 else 0 end) as cnt_cpt_90685,
    sum(case when resource_code = 'cpt_90700'     then 1 else 0 end) as cnt_cpt_90700,
    sum(case when resource_code = 'cpt_90713'     then 1 else 0 end) as cnt_cpt_90713,
    sum(case when resource_code = 'cpt_90716'     then 1 else 0 end) as cnt_cpt_90716,
    sum(case when resource_code = 'cpt_99392'     then 1 else 0 end) as cnt_cpt_99392,
    sum(case when resource_code = 'cpt_90633'     then 1 else 0 end) as cnt_cpt_90633,
    sum(case when resource_code = 'cpt_96110'     then 1 else 0 end) as cnt_cpt_96110,
    sum(case when resource_code = 'cpt_99393'     then 1 else 0 end) as cnt_cpt_99393,
    sum(case when resource_code = 'icd9_V06.4'    then 1 else 0 end) as cnt_icd9_V06_4,
    sum(case when resource_code = 'cpt_90707'     then 1 else 0 end) as cnt_cpt_90707,
       

    
From resource 
Where visit_date < '2017-01-01'
Group by patient_id
"""
lumiata.execute(query)

In [170]:
pd.read_sql_query('SELECT * FROM res_one_hot LIMIT 5', lumiata)

,patient_id,cnt_cpt_83036,cnt_icd9_401_1,cnt_icd10_I10,cnt_icd10_E78_5,cnt_icd9_401_9,cnt_cpt_99223,cnt_cpt_93306,cnt_cpt_71010,cnt_icd9_272_4,...,cnt_cpt_90685,cnt_cpt_90700,cnt_cpt_90713,cnt_cpt_90716,cnt_cpt_99392,cnt_cpt_90633,cnt_cpt_96110,cnt_cpt_99393,cnt_icd9_V06_4,cnt_cpt_90707
0,pat_0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,pat_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,pat_10,0,0,3,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,pat_100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,pat_1000,1,0,4,3,3,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


### Days Features

- days since first/last visit
- days since first/last icd/cpt resource
- days since the most recent high claim rate resource

In [171]:
lumiata.execute('drop table if exists res_days')

In [172]:
# generate features using the resource table

query = """
Create table res_days as 

Select patient_id, 

    -- count days since first/last visit
    max(julianday("2016-12-31") - julianday(visit_date)) as d_first_visit,
    min(julianday("2016-12-31") - julianday(visit_date)) as d_last_visit,

    -- count days since first/last resource
    max(julianday("2016-12-31") - (case when resource_code <> 'None' then julianday(visit_date) else null end)) as d_first_res,
    min(julianday("2016-12-31") - (case when resource_code <> 'None' then julianday(visit_date) else null end)) as d_last_res,

    -- count days since first/last icd resource
    max(julianday("2016-12-31") - (case when resource_code like 'icd%' then julianday(visit_date) else null end)) as d_first_icd,
    min(julianday("2016-12-31") - (case when resource_code like 'icd%' then julianday(visit_date) else null end)) as d_last_icd,

    -- count days since first/last cpt resource
    max(julianday("2016-12-31") - (case when resource_code like 'cpt%' then julianday(visit_date) else null end)) as d_first_cpt,
    min(julianday("2016-12-31") - (case when resource_code like 'cpt%' then julianday(visit_date) else null end)) as d_last_cpt,

    -- count days since the most recent high claim rate resource
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_83036'    then julianday(visit_date) else null end))  as d_last_cpt_83036,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_401.1'   then julianday(visit_date) else null end))  as d_last_icd9_401_1,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_I10'    then julianday(visit_date) else null end))  as d_last_icd10_I10,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_E78.5'  then julianday(visit_date) else null end))  as d_last_icd10_E78_5,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_401.9'   then julianday(visit_date) else null end))  as d_last_icd9_401_9,
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_99223'    then julianday(visit_date) else null end))  as d_last_cpt_99223,
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_93306'    then julianday(visit_date) else null end))  as d_last_cpt_93306,
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_71010'    then julianday(visit_date) else null end))  as d_last_cpt_71010,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_272.4'   then julianday(visit_date) else null end))  as d_last_icd9_272_4,
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_99232'    then julianday(visit_date) else null end))  as d_last_cpt_99232,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_R73.09' then julianday(visit_date) else null end))  as d_last_icd10_R73_09,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_R73.01' then julianday(visit_date) else null end))  as d_last_icd10_R73_01,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_790.29'  then julianday(visit_date) else null end))  as d_last_icd9_790_29,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_I50.9'  then julianday(visit_date) else null end))  as d_last_icd10_I50_9,
    min(julianday('2016-12-31') - (case when resource_code = 'icd10_E66.01' then julianday(visit_date) else null end))  as d_last_icd10_E66_01,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_790.21'  then julianday(visit_date) else null end))  as d_last_icd9_790_21,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_414.01'  then julianday(visit_date) else null end))  as d_last_icd9_414_01,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_278.01'  then julianday(visit_date) else null end))  as d_last_icd9_278_01,
    min(julianday('2016-12-31') - (case when resource_code = 'icd9_428.0'   then julianday(visit_date) else null end))  as d_last_icd9_428_0,
    min(julianday('2016-12-31') - (case when resource_code = 'cpt_A7037'    then julianday(visit_date) else null end))  as d_last_cpt_A7037,
    
    -- count days since first high claim rate resource   
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_83036'    then julianday(visit_date) else null end)) as d_first_cpt_83036,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_401.1'   then julianday(visit_date) else null end)) as d_first_icd9_401_1,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_I10'    then julianday(visit_date) else null end)) as d_first_icd10_I10,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_E78.5'  then julianday(visit_date) else null end)) as d_first_icd10_E78_5,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_401.9'   then julianday(visit_date) else null end)) as d_first_icd9_401_9,
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_99223'    then julianday(visit_date) else null end)) as d_first_cpt_99223,
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_93306'    then julianday(visit_date) else null end)) as d_first_cpt_93306,
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_71010'    then julianday(visit_date) else null end)) as d_first_cpt_71010,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_272.4'   then julianday(visit_date) else null end)) as d_first_icd9_272_4,
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_99232'    then julianday(visit_date) else null end)) as d_first_cpt_99232,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_R73.09' then julianday(visit_date) else null end)) as d_first_icd10_R73_09,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_R73.01' then julianday(visit_date) else null end)) as d_first_icd10_R73_01,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_790.29'  then julianday(visit_date) else null end)) as d_first_icd9_790_29,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_I50.9'  then julianday(visit_date) else null end)) as d_first_icd10_I50_9,
    max(julianday('2016-12-31') - (case when resource_code = 'icd10_E66.01' then julianday(visit_date) else null end)) as d_first_icd10_E66_01,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_790.21'  then julianday(visit_date) else null end)) as d_first_icd9_790_21,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_414.01'  then julianday(visit_date) else null end)) as d_first_icd9_414_01,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_278.01'  then julianday(visit_date) else null end)) as d_first_icd9_278_01,
    max(julianday('2016-12-31') - (case when resource_code = 'icd9_428.0'   then julianday(visit_date) else null end)) as d_first_icd9_428_0,
    max(julianday('2016-12-31') - (case when resource_code = 'cpt_A7037'    then julianday(visit_date) else null end)) as d_first_cpt_A7037

From resource 
Where visit_date < '2017-01-01'
Group by patient_id
"""
lumiata.execute(query)

In [174]:
pd.read_sql_query('SELECT * FROM res_days LIMIT 5', lumiata)

,patient_id,d_first_visit,d_last_visit,d_first_res,d_last_res,d_first_icd,d_last_icd,d_first_cpt,d_last_cpt,d_last_cpt_83036,...,d_first_icd10_R73_09,d_first_icd10_R73_01,d_first_icd9_790_29,d_first_icd10_I50_9,d_first_icd10_E66_01,d_first_icd9_790_21,d_first_icd9_414_01,d_first_icd9_278_01,d_first_icd9_428_0,d_first_cpt_A7037
0,pat_0,1083.0,7.0,955.0,258.0,955.0,258.0,955.0,258.0,NaN,...,None,None,NaN,None,None,None,None,None,None,None
1,pat_1,1116.0,151.0,1116.0,151.0,1116.0,151.0,1116.0,151.0,NaN,...,None,None,NaN,None,None,None,None,None,None,None
2,pat_10,1435.0,178.0,1435.0,178.0,1435.0,178.0,1435.0,178.0,NaN,...,None,None,NaN,None,None,None,None,None,None,None
3,pat_100,938.0,17.0,938.0,17.0,938.0,17.0,938.0,17.0,NaN,...,None,None,NaN,None,None,None,None,None,None,None
4,pat_1000,1083.0,28.0,1072.0,53.0,1072.0,53.0,1072.0,53.0,1072.0,...,None,None,1072.0,None,None,None,None,None,None,None


## Observation Data

- number of unique observations (lab) per patinent with and without time window
- number of observations (lab) per patient with and without time window
- number of lab with valid interpretation with and without time window
- number of date with lab tests with and without time window
- number of observation with interpretstion "high" with and without time window
- number of observation with interpretation "low" with and without time window
- number of observation with interpretation 'normal' with and without time window
- number of observation with non-valid interpretation with and without time window

In [150]:
pd.read_sql_query('SELECT * FROM observation LIMIT 5', lumiata)

,patient_id,lab_date,loinc_code,interpretation,value
0,pat_0,2014-11-13,loinc_13457-7,N,82.0
1,pat_0,2014-11-13,loinc_1742-6,N,21.0
2,pat_0,2014-11-13,loinc_1751-7,L,3.0
3,pat_0,2014-11-13,loinc_1759-0,N,1.0
4,pat_0,2014-11-13,loinc_17861-6,N,9.0


In [206]:
lumiata.execute('drop table if exists obs_num_labs')

In [207]:
# generate features using the resource table

query = """
Create table obs_num_labs as

select patient_id,
    -- number of unique observations (lab) per patinent with and without time window
    count(distinct loinc_code) as num_unique_lab,
    count(distinct case when lab_date >= '2016-12-01' then loinc_code else null end) as num_lab_1mo, 
    count(distinct case when lab_date >= '2016-11-01' then loinc_code else null end) as num_lab_2mo,
    count(distinct case when lab_date >= '2016-10-01' then loinc_code else null end) as num_lab_3mo, 
    count(distinct case when lab_date >= '2016-07-01' then loinc_code else null end) as num_lab_6mo,
    count(distinct case when lab_date >= '2016-01-01' then loinc_code else null end) as num_lab_12mo, 
    
    -- number of observations (lab) per patient with and without time window
    count(loinc_code) as num_total_test,
    count(case when lab_date >= '2016-12-01' then loinc_code else null end) as num_uniq_lab_1mo, 
    count(case when lab_date >= '2016-11-01' then loinc_code else null end) as num_uniq_lab_2mo,
    count(case when lab_date >= '2016-10-01' then loinc_code else null end) as num_uniq_lab_3mo, 
    count(case when lab_date >= '2016-07-01' then loinc_code else null end) as num_uniq_lab_6mo,
    count(case when lab_date >= '2016-01-01' then loinc_code else null end) as num_uniq_lab_12mo,
    
    -- number of lab with valid interpretation with and without time window
    sum(case when interpretation is not null then 1 else 0 end) as num_lab_valid_interp, 
    sum(case when interpretation is not null and lab_date >= '2016-12-01' then 1 else 0 end) as num_lab_valid_interp_1mo, 
    sum(case when interpretation is not null and lab_date >= '2016-11-01' then 1 else 0 end) as num_lab_valid_interp_2mo,
    sum(case when interpretation is not null and lab_date >= '2016-10-01' then 1 else 0 end) as num_lab_valid_interp_3mo,
    sum(case when interpretation is not null and lab_date >= '2016-07-01' then 1 else 0 end) as num_lab_valid_interp_6mo,
    sum(case when interpretation is not null and lab_date >= '2016-01-01' then 1 else 0 end) as num_lab_valid_interp_12mo,
    
    -- number of date with lab tests with and without time window
    count(distinct lab_date) as num_labdate,
    count(distinct case when lab_date >= '2016-12-01' then lab_date else null end) as num_labdate_1mo, 
    count(distinct case when lab_date >= '2016-11-01' then lab_date else null end) as num_labdate_2mo,
    count(distinct case when lab_date >= '2016-10-01' then lab_date else null end) as num_labdate_3mo, 
    count(distinct case when lab_date >= '2016-07-01' then lab_date else null end) as num_labdate_6mo,
    count(distinct case when lab_date >= '2016-01-01' then lab_date else null end) as num_labdate_12mo, 
    
    -- number of observation with interpretstion "high" with and without time window
    sum(case when interpretation = 'H' then 1 else 0 end) as num_high_interp,
    sum(case when interpretation = 'H' and lab_date >= '2016-12-01' then 1 else 0 end) as num_high_interp_1mo, 
    sum(case when interpretation = 'H' and lab_date >= '2016-11-01' then 1 else 0 end) as num_high_interp_2mo,
    sum(case when interpretation = 'H' and lab_date >= '2016-10-01' then 1 else 0 end) as num_high_interp_3mo, 
    sum(case when interpretation = 'H' and lab_date >= '2016-07-01' then 1 else 0 end) as num_high_interp_6mo,
    sum(case when interpretation = 'H' and lab_date >= '2016-01-01' then 1 else 0 end) as num_high_interp_12mo,
       
    -- number of observation with interpretation "low" with and without time window
    sum(case when interpretation = 'L' then 1 else 0 end) as num_low_interp,
    sum(case when interpretation = 'L' and lab_date >= '2016-12-01' then 1 else 0 end) as num_low_interp_1mo, 
    sum(case when interpretation = 'L' and lab_date >= '2016-11-01' then 1 else 0 end) as num_low_interp_2mo,
    sum(case when interpretation = 'L' and lab_date >= '2016-10-01' then 1 else 0 end) as num_low_interp_3mo, 
    sum(case when interpretation = 'L' and lab_date >= '2016-07-01' then 1 else 0 end) as num_low_interp_6mo,
    sum(case when interpretation = 'L' and lab_date >= '2016-01-01' then 1 else 0 end) as num_low_interp_12mo,
    
    --- number of observation with interpretation 'normal' with and without time window
    sum(case when interpretation = 'N' then 1 else 0 end) as num_norm_interp,
    sum(case when interpretation = 'N' and lab_date >= '2016-12-01' then 1 else 0 end) as num_norm_interp_1mo, 
    sum(case when interpretation = 'N' and lab_date >= '2016-11-01' then 1 else 0 end) as num_norm_interp_2mo,
    sum(case when interpretation = 'N' and lab_date >= '2016-10-01' then 1 else 0 end) as num_norm_interp_3mo, 
    sum(case when interpretation = 'N' and lab_date >= '2016-07-01' then 1 else 0 end) as num_norm_interp_6mo,
    sum(case when interpretation = 'N' and lab_date >= '2016-01-01' then 1 else 0 end) as num_norm_interp_12mo,
    
    -- number of observation with non-valid interpretation with and without time window
    sum(case when interpretation is null then 1 else 0 end) as num_none_interp, 
    sum(case when interpretation is null and lab_date >= '2016-12-01' then 1 else 0 end) as num_none_interp_1mo, 
    sum(case when interpretation is null and lab_date >= '2016-11-01' then 1 else 0 end) as num_none_interp_2mo,
    sum(case when interpretation is null and lab_date >= '2016-10-01' then 1 else 0 end) as num_none_interp_3mo, 
    sum(case when interpretation is null and lab_date >= '2016-07-01' then 1 else 0 end) as num_none_interp_6mo,
    sum(case when interpretation is null and lab_date >= '2016-01-01' then 1 else 0 end) as num_none_interp_12mo
    
    
from observation
where lab_date < '2017-01-01'
Group by patient_id; 
"""
lumiata.execute(query)

In [177]:
pd.read_sql_query('SELECT * FROM obs_num_labs LIMIT 5', lumiata)

,patient_id,num_unique_lab,num_uniq_lab_1mo,num_uniq_lab_2mo,num_uniq_lab_3mo,num_uniq_lab_6mo,num_uniq_lab_12mo,num_total_test,num_uniq_lab_1mo:1,num_uniq_lab_2mo:1,...,num_norm_interp_2mo,num_norm_interp_3mo,num_norm_interp_6mo,num_norm_interp_12mo,num_none_interp,num_none_interp_1mo,num_none_interp_2mo,num_none_interp_3mo,num_none_interp_6mo,num_none_interp_12mo
0,pat_0,60,0,0,0,0,0,60,0,0,...,0,0,0,0,16,0,0,0,0,0
1,pat_1,4,0,0,0,0,4,7,0,0,...,0,0,0,1,1,0,0,0,0,1
2,pat_100,35,0,0,0,0,0,35,0,0,...,0,0,0,0,2,0,0,0,0,0
3,pat_1000,58,0,0,40,40,41,146,0,0,...,0,25,25,55,13,0,0,1,1,2
4,pat_10000,31,28,28,28,28,29,75,28,28,...,21,21,21,34,2,1,1,1,1,1


### Check Claim Rate by Lab Code

In [133]:
lumiata.execute('drop table if exists ob_freq')

In [134]:
# generate features using the resource table

query = """
Create table ob_freq as 

-- claim rate for each lab / loinc_code

Select loinc_code, 
       count(distinct r.patient_id) as num_pat,
       count(distinct case when b.tag_dm2 is not null then b.patient_id else null end) as cnt_claim_pat,
       count(distinct case when b.tag_dm2 is not null then b.patient_id else null end)*1.0 / count(distinct r.patient_id) as claim_rate,
       
       count(distinct case when interpretation = 'H' then b.patient_id else null end) as cnt_pat_H,
       count(distinct case when b.tag_dm2 is not null and interpretation = 'H' then b.patient_id else null end)*1.0 
       / count(distinct case when interpretation = 'H' then b.patient_id else null end) as claim_rate_H,
       
       count(distinct case when interpretation = 'L' then b.patient_id else null end) as cnt_pat_L,
       count(distinct case when b.tag_dm2 is not null and interpretation = 'L' then b.patient_id else null end)*1.0 
       / count(distinct case when interpretation = 'L' then b.patient_id else null end) as claim_rate_L,
      
       count(distinct case when interpretation = 'N' then b.patient_id else null end) as cnt_pat_N,
       count(distinct case when b.tag_dm2 is not null and interpretation = 'N' then b.patient_id else null end)*1.0 
       / count(distinct case when interpretation = 'N' then b.patient_id else null end) as claim_rate_N,

       count(distinct case when interpretation is null then b.patient_id else null end) as cnt_pat_None,
       count(distinct case when b.tag_dm2 is not null and interpretation is null then b.patient_id else null end)*1.0 
       / count(distinct case when interpretation is null then b.patient_id else null end) as claim_rate_None
      

From observation r 
left join basic_info b on r.patient_id = b.patient_id

Where lab_date < '2017-01-01'
      and b.split = 'train'
      and (b.tag_dm2 <= '2017-12-31' or b.tag_dm2 is null)
      
Group by loinc_code
order by cnt_claim_pat desc
"""
lumiata.execute(query)

In [215]:
pd.read_sql_query('SELECT * FROM ob_freq where num_pat > 500 order by claim_rate desc LIMIT 10', lumiata)

,loinc_code,num_pat,cnt_claim_pat,claim_rate,cnt_pat_H,claim_rate_H,cnt_pat_L,claim_rate_L,cnt_pat_N,claim_rate_N,cnt_pat_None,claim_rate_None
0,loinc_14957-5,769,126,0.163849,77,0.298701,223,0.156951,480,0.168750,56,0.107143
1,loinc_9318-7,576,94,0.163194,76,0.171053,0,NaN,513,0.161793,12,0.250000
2,loinc_27353-2,2049,234,0.114202,55,0.581818,1,0.000000,1998,0.102102,0,NaN
3,loinc_2161-8,1337,148,0.110696,67,0.134328,495,0.072727,928,0.120690,1,0.000000
4,loinc_30934-4,1059,114,0.107649,357,0.134454,9,0.333333,751,0.091877,0,NaN
5,loinc_4548-4,5792,562,0.097030,9,1.000000,11,0.090909,5777,0.095724,0,NaN
6,loinc_1971-1,628,60,0.095541,49,0.102041,589,0.095076,2,0.000000,0,NaN
7,loinc_2865-4,534,46,0.086142,0,NaN,519,0.086705,13,0.076923,2,0.000000
8,loinc_2868-8,534,46,0.086142,1,0.000000,486,0.082305,62,0.112903,2,0.000000
9,loinc_2874-6,535,46,0.085981,30,0.100000,238,0.079832,297,0.097643,2,0.000000


In [216]:
lumiata.execute('drop table if exists ob_one_hot')

In [217]:
# generate features using the resource table

query = """
Create table ob_one_hot as 

Select 
    patient_id, 

    -- avg value of loinc_code with high claim rate and num_patient > 1000
    avg(case when loinc_code = 'loinc_27353-2' then value else null end) as avg_loinc_27353_2,
    avg(case when loinc_code = 'loinc_2161-8'  then value else null end) as avg_loinc_2161_8,
    avg(case when loinc_code = 'loinc_30934-4' then value else null end) as avg_loinc_30934_4,
    avg(case when loinc_code = 'loinc_4548-4'  then value else null end) as avg_loinc_4548_4,
    avg(case when loinc_code = 'loinc_17856-6' then value else null end) as avg_loinc_17856_6,
    avg(case when loinc_code = 'loinc_10839-9' then value else null end) as avg_loinc_10839_9,
    avg(case when loinc_code = 'loinc_14979-9' then value else null end) as avg_loinc_14979_9,
    avg(case when loinc_code = 'loinc_2157-6'  then value else null end) as avg_loinc_2157_6,
    avg(case when loinc_code = 'loinc_2857-1'  then value else null end) as avg_loinc_2857_1,
    avg(case when loinc_code = 'loinc_5902-2'  then value else null end) as avg_loinc_5902_2,
    
    -- avg value of loinc_code with high claim rate and num_patient > 500
    avg(case when loinc_code = 'loinc_14957-5' then value else null end) as avg_loinc_14957_5,
    avg(case when loinc_code = 'loinc_9318-7'  then value else null end) as avg_loinc_9318_7,
    avg(case when loinc_code = 'loinc_1971-1'  then value else null end) as avg_loinc_1971_1,
    avg(case when loinc_code = 'loinc_2865-4'  then value else null end) as avg_loinc_2865_4,
    avg(case when loinc_code = 'loinc_2868-8'  then value else null end) as avg_loinc_2868_8
       

From observation 
Where lab_date < '2017-01-01'
Group by patient_id
"""
lumiata.execute(query)

In [15]:
lumiata.execute('drop table if exists last_ob')

In [16]:
# generate features using the resource table

query = """
Create table last_ob as 

Select l.patient_id as patient_id, 
       l.loinc_code as loinc_code,
       l.lab_date as lab_date,
       l.interpretation as interpretation,
       l.value as value
    
    
From observation l
left join observation r 
on l.patient_id = r.patient_id and l.loinc_code = r.loinc_code and l.lab_date < r.lab_date

Where l.lab_date < '2017-01-01'
and r.lab_date is null

"""
lumiata.execute(query)

In [17]:
pd.read_sql_query('SELECT * FROM last_ob LIMIT 5', lumiata)

,patient_id,loinc_code,lab_date,interpretation,value
0,pat_0,loinc_13457-7,2014-11-13,N,82.0
1,pat_0,loinc_1742-6,2014-11-13,N,21.0
2,pat_0,loinc_1751-7,2014-11-13,L,3.0
3,pat_0,loinc_1759-0,2014-11-13,N,1.0
4,pat_0,loinc_17861-6,2014-11-13,N,9.0


In [18]:
pd.read_sql_query('SELECT * FROM observation where loinc_code = "loinc_2865-4" and lab_date < "2017-01-01" and patient_id = "pat_89"', lumiata)

,patient_id,lab_date,loinc_code,interpretation,value
0,pat_89,2016-07-16,loinc_2865-4,L,0.0
1,pat_89,2014-10-09,loinc_2865-4,L,0.0


In [20]:
pd.read_sql_query('SELECT * FROM last_ob where patient_id = "pat_89" and loinc_code = "loinc_2865-4"', lumiata)

,patient_id,loinc_code,lab_date,interpretation,value
0,pat_89,loinc_2865-4,2016-07-16,L,0.0


In [25]:
lumiata.execute('drop table if exists last_ob_one_hot')

In [26]:
# generate features using the resource table

query = """
Create table last_ob_one_hot as 

Select 
    patient_id, 
    
    -- value of last loinc_code 
    avg(case when loinc_code = 'loinc_27353-2' then value else null end) as v_last_loinc_27353_2,
    avg(case when loinc_code = 'loinc_2161-8'  then value else null end) as v_last_loinc_2161_8,
    avg(case when loinc_code = 'loinc_30934-4' then value else null end) as v_last_loinc_30934_4,
    avg(case when loinc_code = 'loinc_4548-4'  then value else null end) as v_last_loinc_4548_4,
    avg(case when loinc_code = 'loinc_17856-6' then value else null end) as v_last_loinc_17856_6,
    avg(case when loinc_code = 'loinc_10839-9' then value else null end) as v_last_loinc_10839_9,
    avg(case when loinc_code = 'loinc_14979-9' then value else null end) as v_last_loinc_14979_9,
    avg(case when loinc_code = 'loinc_2157-6'  then value else null end) as v_last_loinc_2157_6,
    avg(case when loinc_code = 'loinc_2857-1'  then value else null end) as v_last_loinc_2857_1,
    avg(case when loinc_code = 'loinc_5902-2'  then value else null end) as v_last_loinc_5902_2,
    avg(case when loinc_code = 'loinc_14957-5' then value else null end) as v_last_loinc_14957_5,
    avg(case when loinc_code = 'loinc_9318-7'  then value else null end) as v_last_loinc_9318_7,
    avg(case when loinc_code = 'loinc_1971-1'  then value else null end) as v_last_loinc_1971_1,
    avg(case when loinc_code = 'loinc_2865-4'  then value else null end) as v_last_loinc_2865_4,
    avg(case when loinc_code = 'loinc_2868-8'  then value else null end) as v_last_loinc_2868_8

    
    -- interpretation of last loinc_code 
    --(case when loinc_code = 'loinc_27353-2' then interpretation else null end) as int_last_loinc_27353_2,
    --(case when loinc_code = 'loinc_2161-8'  then interpretation else null end) as int_last_loinc_2161_8,
    --(case when loinc_code = 'loinc_30934-4' then interpretation else null end) as int_last_loinc_30934_4,
    --(case when loinc_code = 'loinc_4548-4'  then interpretation else null end) as int_last_loinc_4548_4,
    --(case when loinc_code = 'loinc_17856-6' then interpretation else null end) as int_last_loinc_17856_6,
    --(case when loinc_code = 'loinc_10839-9' then interpretation else null end) as int_last_loinc_10839_9,
    --(case when loinc_code = 'loinc_14979-9' then interpretation else null end) as int_last_loinc_14979_9,
    --(case when loinc_code = 'loinc_2157-6'  then interpretation else null end) as int_last_loinc_2157_6,
    --(case when loinc_code = 'loinc_2857-1'  then interpretation else null end) as int_last_loinc_2857_1,
    --(case when loinc_code = 'loinc_5902-2'  then interpretation else null end) as int_last_loinc_5902_2,
    --(case when loinc_code = 'loinc_6301-6'  then interpretation else null end) as int_last_loinc_6301_6,
    --(case when loinc_code = 'loinc_3084-1'  then interpretation else null end) as int_last_loinc_3084_1,
    --(case when loinc_code = 'loinc_2284-8'  then interpretation else null end) as int_last_loinc_2284_8,
    --(case when loinc_code = 'loinc_3026-2'  then interpretation else null end) as int_last_loinc_3026_2,
    --(case when loinc_code = 'loinc_2777-1'  then interpretation else null end) as int_last_loinc_2777_1,
    --(case when loinc_code = 'loinc_43305-2' then interpretation else null end) as int_last_loinc_43305_2
    
    
From last_ob 
Where lab_date < '2017-01-01'

Group by patient_id
"""
lumiata.execute(query)

In [27]:
pd.read_sql_query('SELECT * FROM last_ob_one_hot where patient_id = "pat_89" LIMIT 5', lumiata)

,patient_id,v_last_loinc_27353_2,v_last_loinc_2161_8,v_last_loinc_30934_4,v_last_loinc_4548_4,v_last_loinc_17856_6,v_last_loinc_10839_9,v_last_loinc_14979_9,v_last_loinc_2157_6,v_last_loinc_2857_1,v_last_loinc_5902_2,v_last_loinc_14957_5,v_last_loinc_9318_7,v_last_loinc_1971_1,v_last_loinc_2865_4,v_last_loinc_2868_8
0,pat_89,None,None,None,None,None,None,31.0,77.0,None,11.0,None,None,None,0.0,1.0
